<a href="https://colab.research.google.com/github/a123r123p/Documents/blob/main/VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

Imports

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load the MNIST dataset

In [3]:
# Load the MNIST dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize the images to the range [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# Reshape the data to include the channel dimension
x_train = x_train.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

# Print the shapes of the datasets
print("Training data shape:", x_train.shape)
print("Training labels shape:", y_train.shape)
print("Testing data shape:", x_test.shape)
print("Testing labels shape:", y_test.shape)


11490434/11490434 [==============================] - 0s 0us/step
Training data shape: (60000, 28, 28, 1)
Training labels shape: (60000,)
Testing data shape: (10000, 28, 28, 1)
Testing labels shape: (10000,)


# Define the encoder model

In [4]:
# Define the encoder model
def build_encoder(input_shape):
    encoder = models.Sequential()
    encoder.add(layers.InputLayer(input_shape=input_shape))
    encoder.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    encoder.add(layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    encoder.add(layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    encoder.add(layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    encoder.add(layers.MaxPooling2D((2, 2), padding='same'))
    encoder.add(layers.Flatten())
    encoder.add(layers.Dense(256, activation='relu'))

    return encoder

# Input shape for the MNIST dataset (28x28 grayscale images)
input_shape = (28, 28, 1)
encoder = build_encoder(input_shape)

# Print the encoder model summary
# encoder.summary()

# Define the decoder model

In [ ]:
# # Define the decoder model
# def build_decoder(encoded_shape):
#     decoder = models.Sequential()
#     decoder.add(layers.InputLayer(input_shape=encoded_shape))
#     decoder.add(layers.Dense(4 * 4 * 128, activation='relu'))
#     decoder.add(layers.Reshape((4, 4, 128)))
#     decoder.add(layers.Conv2DTranspose(128, (3, 3), activation='relu', padding='same'))
#     decoder.add(layers.UpSampling2D((2, 2)))
#     decoder.add(layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'))
#     decoder.add(layers.UpSampling2D((2, 2)))
#     decoder.add(layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same'))
#     decoder.add(layers.UpSampling2D((2, 2)))
#     decoder.add(layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same'))

#     return decoder

# # Encoded shape from the encoder's output (assuming the final dense layer in the encoder has 256 units)
# encoded_shape = (256,)
# decoder = build_decoder(encoded_shape)

# # Print the decoder model summary
# decoder.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 6272)              1611904   
                                                                 
 reshape_5 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 conv2d_transpose_18 (Conv2  (None, 7, 7, 128)         147584    
 DTranspose)                                                     
                                                                 
 up_sampling2d_13 (UpSampli  (None, 14, 14, 128)       0         
 ng2D)                                                           
                                                                 
 conv2d_transpose_19 (Conv2  (None, 14, 14, 64)        73792     
 DTranspose)                                                     
                                                     

In [5]:
# Modify the decoder to output images of shape (28, 28, 1)
def build_decoder(encoded_shape):
    decoder = models.Sequential()
    decoder.add(layers.InputLayer(input_shape=encoded_shape))
    decoder.add(layers.Dense(7 * 7 * 128, activation='relu'))  # Adjusted to match the encoded output size after max pooling
    decoder.add(layers.Reshape((7, 7, 128)))  # Reshape according to the last pooling output
    decoder.add(layers.Conv2DTranspose(64, (3, 3), activation='relu', padding='same'))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2DTranspose(32, (3, 3), activation='relu', padding='same'))
    decoder.add(layers.UpSampling2D((2, 2)))
    decoder.add(layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same'))

    return decoder

# Encoded shape from the encoder's output (assuming the final dense layer in the encoder has 256 units)
encoded_shape = (256,)
decoder = build_decoder(encoded_shape)

# Print the decoder model summary
decoder.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 6272)              1611904   
                                                                 
 reshape (Reshape)           (None, 7, 7, 128)         0         
                                                                 
 conv2d_transpose (Conv2DTr  (None, 7, 7, 64)          73792     
 anspose)                                                        
                                                                 
 up_sampling2d (UpSampling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                                 
 conv2d_transpose_1 (Conv2D  (None, 14, 14, 32)        18464     
 Transpose)                                                      
                                                      

# Full autoencoder model

In [6]:
# Full autoencoder model
autoencoder_input = layers.Input(shape=(28, 28, 1))
encoded_output = encoder(autoencoder_input)
decoded_output = decoder(encoded_output)

print("Input shape:", autoencoder_input.shape)
print("Encoded shape:", encoded_output.shape)
print("Decoded shape:", decoded_output.shape)

print("Input :", autoencoder_input)
print("Encoded :", encoded_output)
print("Decoded :", decoded_output)

autoencoder = models.Model(autoencoder_input, decoded_output)

# Print the full autoencoder model summary
# autoencoder.summary()

# Compile the autoencoder model
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder model
history = autoencoder.fit(x_train, x_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the autoencoder on the test data
test_loss = autoencoder.evaluate(x_test, x_test)
print(f'Test loss: {test_loss}')

Input shape: (None, 28, 28, 1)
Encoded shape: (None, 256)
Decoded shape: (None, 28, 28, 1)
Input : KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_3'), name='input_3', description="created by layer 'input_3'")
Encoded : KerasTensor(type_spec=TensorSpec(shape=(None, 256), dtype=tf.float32, name=None), name='sequential/dense/Relu:0', description="created by layer 'sequential'")
Decoded : KerasTensor(type_spec=TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), name='sequential_1/conv2d_transpose_2/Sigmoid:0', description="created by layer 'sequential_1'")
Epoch 1/10
1500/1500 [==============================] - 16s 6ms/step - loss: 0.1251 - val_loss: 0.0910
Epoch 2/10
1500/1500 [==============================] - 10s 7ms/step - loss: 0.0848 - val_loss: 0.0814
Epoch 3/10
1500/1500 [==============================] - 10s 6ms/step - loss: 0.0793 - val_loss: 0.0790
Epoch 4/10
1500/1500 [==============================] - 10s 7ms/step - loss: 